# Iterated Prisoner's Dilemma On A Network

## Imports and Config

In [ ]:
import math
import random
import logging
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
from dataclasses import dataclass
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib.ticker import PercentFormatter
from matplotlib.animation import FuncAnimation

In [ ]:
%matplotlib notebook
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
)

## Matrices

In [ ]:
payoff_matrices = {  # some of these don't look right
    "Default": {
        ("C", "C"): (3, 3),
        ("C", "D"): (0, 5),
        ("D", "C"): (5, 0),
        ("D", "D"): (0, 0),
    },
    "Canonical": {
        ("C", "C"): (-1, -1),
        ("C", "D"): (-3, 0),
        ("D", "C"): (0, -3),
        ("D", "D"): (-2, -2),
    },
    "Friend or Foe": {
        ("C", "C"): (1, 1),
        ("C", "D"): (0, 2),
        ("D", "C"): (2, 0),
        ("D", "D"): (0, 0),
    },
    "Snowdrift": {
        ("C", "C"): (500, 500),
        ("C", "D"): (200, 800),
        ("D", "C"): (800, 200),
        ("D", "D"): (0, 0),
    },
    "Prisoners": {
        ("C", "C"): (500, 500),
        ("C", "D"): (-200, 1200),
        ("D", "C"): (1200, -200),
        ("D", "D"): (0, 0),
    },
}

## Classes

In [ ]:
from Prison import ActionStrategy, ImitationStrategy, FermiStrategy, ReinforcementLearningStrategy, TitForTatStrategy
from Prison import Agent, Network, NetworkSimulation
from Prison import experiment

## Visualization

In [ ]:
# def experiment(
#     model_class,
#     strategy_class,
#     strategy_kwargs=None,
#     steps=100,
#     seed=42,
#     interval=300,
#     payoff_matrix=payoff_matrices["Default"],
#     title="",
#     kind="grid",
#     n=400,
#     is_grid=False,
#     **graph_kwargs,
# ):
#     """
#     Produce animations showing the network state over time.
#     """
#     payoff_matrix = payoff_matrix
#     strategy_kwargs = strategy_kwargs or {}
#     model = model_class(
#         kind=kind,
#         n=n,
#         seed=seed,
#         rounds=steps,
#         payoff_matrix=payoff_matrix,
#         strategy=strategy_class,
#         strategy_kwargs=strategy_kwargs,
#         **graph_kwargs,
#     )

#     graph = model.graph
#     n_nodes = graph.number_of_nodes()

#     C_COOP, C_DEFECT = "#40B0A6", "#FFBE6A"
#     cmap = ListedColormap([C_COOP, C_DEFECT])
#     fig, (ax_sim, ax_stats) = plt.subplots(
#         2, 1, figsize=(7, 8), gridspec_kw={"height_ratios": [4, 1]}
#     )

#     # -------------------------
#     # Stats plot (C% and D%)
#     # -------------------------
#     xs, ys_c, ys_d = [], [], []

#     (line_c,) = ax_stats.plot([], [], lw=2, label="C")
#     (line_d,) = ax_stats.plot([], [], lw=2, label="D")

#     ax_stats.set_xlim(0, steps)
#     ax_stats.set_ylim(0, 100)
#     ax_stats.yaxis.set_major_formatter(PercentFormatter(xmax=100))
#     ax_stats.set_ylabel("Population")
#     ax_stats.grid(True, linestyle=":", alpha=0.4)
#     ax_stats.legend(frameon=False, ncol=2, loc="upper right")

#     # -------------------------
#     # Simulation plot
#     # -------------------------
#     if is_grid:
#         dim = int(math.isqrt(n_nodes))
#         if dim * dim != n_nodes:
#             raise ValueError(f"Grid mode needs square number of nodes, got {n_nodes}.")

#         def state_as_grid():
#             state = model._get_state()
#             grid = [[0] * dim for _ in range(dim)]
#             for node, val in state.items():
#                 grid[node // dim][node % dim] = val
#             return grid

#         sim_artist = ax_sim.imshow(state_as_grid(), cmap=cmap, vmin=0, vmax=1)
#         ax_sim.set_xticks([])
#         ax_sim.set_yticks([])

#         def update_sim():
#             sim_artist.set_data(state_as_grid())

#     else:
#         pos = nx.spring_layout(graph, seed=seed)
#         nodelist = list(graph.nodes())
#         nx.draw_networkx_edges(graph, pos, ax=ax_sim, alpha=0.3, edge_color="gray")
#         state0 = model._get_state()
#         sim_artist = nx.draw_networkx_nodes(
#             graph,
#             pos,
#             nodelist=nodelist,
#             node_color=[state0[i] for i in nodelist],
#             cmap=cmap,
#             vmin=0,
#             vmax=1,
#             node_size=80,
#             edgecolors="gray",
#             ax=ax_sim,
#         )
#         ax_sim.axis("off")

#         def update_sim():
#             state = model._get_state()
#             sim_artist.set_array([state[i] for i in nodelist])

#     # -------------------------
#     # Animation update
#     # -------------------------
#     def update(frame):
#         if frame > 0:
#             model.step()

#         ax_sim.set_title(f"{title} (Step {frame}/{steps})")

#         update_sim()

#         state = model._get_state()
#         d = sum(state.values())
#         c = n_nodes - d

#         xs.append(frame)
#         ys_c.append(100 * c / n_nodes)
#         ys_d.append(100 * d / n_nodes)

#         line_c.set_data(xs, ys_c)
#         line_d.set_data(xs, ys_d)

#         return sim_artist, line_c, line_d

#     anim = FuncAnimation(
#         fig,
#         update,
#         frames=steps + 1,
#         interval=interval,
#         blit=True,
#         repeat=False,
#     )
#     return anim

## Experiments

In [ ]:
matrix_names = ["Snowdrift"]
size = 10
n = size * size

runs = [
    # ("Imitation", ImitationStrategy, {}),
    (
        "Reinforcement Learning",
        ReinforcementLearningStrategy,
        {
            "learning_rate": 0.1,
            "epsilon": 0.1,
            "initial_q": 0.0,
        },
    ),
]

graph_setups = [
    # ("Grid", "grid", {}, True),
    ("Small-World", "watts_strogatz", {"k": 4, "p": 0.1}, False),
    ("Erdos-Renyi", "erdos_renyi", {"p": 0.1}, False),
]

for matrix_name in matrix_names:
    matrix = payoff_matrices[matrix_name]

    for graph_label, kind, graph_kwargs, is_grid in graph_setups:
        for strat_label, strat_cls, strat_kwargs in runs:
            ani = experiment(
                network_simulation=NetworkSimulation,
                strategy_class=strat_cls,
                strategy_kwargs=strat_kwargs,
                steps=60,
                seed=42,
                interval=50,
                payoff_matrix=matrix,
                kind=kind,
                n=n,
                is_grid=is_grid,
                title=f"{strat_label} on {matrix_name} ({graph_label})",
                save_gif=False,
                **graph_kwargs,
            )
            display(ani)

## WIP

In [ ]:
df = run_many(
    kind="watts_strogatz",
    n=400,
    payoff_matrix=payoff_matrices["Snowdrift"],
    strategy_class=ImitationStrategy,
    strategy_kwargs={},
    seeds=range(30),
    max_steps=1500,
    graph_kwargs={"k": 4, "p": 0.1},
)
print(df.head())
print(
    df.groupby("attractor")[
        [
            "period",
            "cycle_mean_coop_frac",
            "cycle_mean_largest_cluster",
            "frac_oscillating",
        ]
    ].agg(["mean", "std", "median"])
)

In [ ]:
# Cooperation over time + cluster size distribution
model = NetworkSimulation(
    kind="watts_strogatz",
    n=400,
    seed=1,
    rounds=300,
    payoff_matrix=payoff_matrices["Snowdrift"],
    strategy=ImitationStrategy,
    strategy_kwargs={},
    store_snapshots=False,
    history_window=5,
    store_history=True,
    k=4,
    p=0.1,
)

time_metrics, size_dist = track_cooperation_over_time(
    model, steps=300, sample_every=2, max_cluster_size=40
)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
ax1.plot(time_metrics["t"], time_metrics["coop_frac"], label="Coop fraction")
ax1.plot(
    time_metrics["t"],
    time_metrics["largest_coop_cluster"],
    label="Largest coop cluster",
)
ax1.set_ylabel("Value")
ax1.grid(True, linestyle=":", alpha=0.4)
ax1.legend(frameon=False)

if not size_dist.empty:
    pivot = size_dist.pivot_table(
        index="t", columns="size", values="count", aggfunc="sum", fill_value=0
    )
    ax2.stackplot(pivot.index, pivot.values.T, labels=pivot.columns)
    ax2.set_ylabel("Cluster count")
    ax2.set_xlabel("t")
    ax2.grid(True, linestyle=":", alpha=0.4)

plt.tight_layout()
plt.show()

# Batch summary plots over seeds
batch_df = run_many(
    kind="watts_strogatz",
    n=400,
    payoff_matrix=payoff_matrices["Snowdrift"],
    strategy_class=ImitationStrategy,
    strategy_kwargs={},
    seeds=range(30),
    max_steps=1500,
    graph_kwargs={"k": 4, "p": 0.1},
)

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
axes[0].hist(batch_df["period"].dropna(), bins=20)
axes[0].set_title("Period")
axes[1].hist(batch_df["cycle_mean_coop_frac"].dropna(), bins=20)
axes[1].set_title("Cycle mean coop")
axes[2].hist(batch_df["cycle_mean_largest_cluster"].dropna(), bins=20)
axes[2].set_title("Cycle mean largest cluster")
for ax in axes:
    ax.grid(True, linestyle=":", alpha=0.4)
plt.tight_layout()
plt.show()